# Hello World
pr107 is the only problem so far for those <= 5000 including Metric Explicit and Metric Explicit Fixed. None of the 5k-10k problems which are node coords satisfy. 

In [21]:

import tsplib95
from pathlib import Path
import itertools
import numpy as np
from metric_split import best_bipartition
import threading
from concurrent.futures import ThreadPoolExecutor

from utils import is_metric

TSP_LIB_PATH = Path("../../ALL_tsp")
DIMENSION_THRESHOLD = 1000
METRIC_THRESHOLD = min(1000, DIMENSION_THRESHOLD)
METRIC_CHECK = True
METRIC_FIX = False
TOL=1e-2

In [22]:
all_problems : list[tsplib95.models.StandardProblem] = []

for file in sorted(TSP_LIB_PATH.iterdir()): # Loop through every tsp
    if file.suffix != ".tsp" or not file.is_file():
        continue
    problem = tsplib95.load(f"{file.absolute()}")
    if problem.edge_weight_type == "EXPLICIT":
        if problem.dimension > DIMENSION_THRESHOLD or not problem.is_complete():
            continue
        if METRIC_CHECK and not is_metric(dist_mat := problem._create_explicit_matrix().to_numpy(), tol=TOL):
            if METRIC_FIX and problem.is_full_matrix() and is_metric((dist_mat + dist_mat.T) / 2, tol=TOL):
                print(f"Fixing {problem.name} since it could be made metric by symmetrizing")
                problem.edge_weight_type = "EXPLICIT2"
            else:
                continue
        print(f"Adding {problem.name} since it's metric")
    
    all_problems.append(problem)
    print(f"Added {problem.name}")

print("Found", len(all_problems), "euclidean TSPs")

Adding 100001 since it's metric
Added 100001
Adding 10007 since it's metric
Added 10007
Adding 10008 since it's metric
Added 10008
Adding 10010 since it's metric
Added 10010
Adding 11675 since it's metric
Added 11675
Adding 12290 since it's metric
Added 12290
Adding 14850 since it's metric
Added 14850
Adding 15002 since it's metric
Added 15002
Adding 15005 since it's metric
Added 15005
Adding 15007 since it's metric
Added 15007
Adding 16038 since it's metric
Added 16038
Adding 20004 since it's metric
Added 20004
Adding 20007 since it's metric
Added 20007
Adding 20009 since it's metric
Added 20009
Adding 20181 since it's metric
Added 20181
Adding 25001 since it's metric
Added 25001
Adding 25004 since it's metric
Added 25004
Adding 25006 since it's metric
Added 25006
Adding 30001 since it's metric
Added 30001
Adding 30003 since it's metric
Added 30003
Adding 30005 since it's metric
Added 30005
Adding 33001 since it's metric
Added 33001
Adding 35002 since it's metric
Added 35002
Adding 35

In [32]:
def geo_based_dist_matrix(nodes_array, radius: float = 6378.388) -> np.ndarray:
    lat = np.deg2rad(nodes_array[:, 0])
    lon = np.deg2rad(nodes_array[:, 1])

    # Broadcast to pairwise differences
    lat_i = lat[:, None]
    lat_j = lat[None, :]
    lon_i = lon[:, None]
    lon_j = lon[None, :]

    q1 = np.cos(lon_i - lon_j)
    q2 = np.cos(lat_i - lat_j)
    q3 = np.cos(lat_i + lat_j)

    return radius * np.arccos(0.5 * ((1 + q1) * q2 - (1 - q1) * q3))

def convert_to_dist_matrix(problem: tsplib95.models.StandardProblem) -> np.ndarray:
    """Convert a tsplib95 StandardProblem to a distance matrix."""
    n = int(problem.dimension)  # Cast to int to resolve type mismatch
    dist_matrix = np.zeros((n, n))
    nodes_array = np.array(list(problem.node_coords.values()))
    if problem.edge_weight_type in ["EUC_2D", "CEIL_2D", "ATT"]:
        dist_matrix = np.linalg.norm(nodes_array[:, np.newaxis] - nodes_array[np.newaxis, :], axis=2)
        if problem.edge_weight_type == "ATT":
            dist_matrix /= np.sqrt(10)
    elif "GEO" in problem.edge_weight_type:
        dist_matrix = geo_based_dist_matrix(nodes_array)
    elif "EXPLICIT" in problem.edge_weight_type:
        dist_matrix = problem._create_explicit_matrix().to_numpy()
    elif "EXPLICIT2" in problem.edge_weight_type:
        dist_matrix = (dist_matrix + dist_matrix.T) / 2
    else:
        raise ValueError(f"Unsupported edge weight type: {problem.edge_weight_type}")
    if "CEIL" in problem.edge_weight_type:
        dist_matrix = np.ceil(dist_matrix)
    return dist_matrix


all_dist_matrices = ((problem.name, convert_to_dist_matrix(problem)) for problem in all_problems if int(problem.dimension) <= DIMENSION_THRESHOLD)

In [33]:
forest_amounts: dict[str, tuple[int, int] | None] = {}

def _compute_split(args: tuple[str, np.ndarray]):
    name, dist_matrix = args
    #print(name, dist_matrix.shape[0])
    forest = best_bipartition(dist_matrix, s=0.55, tol=TOL)
    return name, None if forest is None else tuple(len(cluster) for cluster in forest)
    
# Run splits concurrently; order preserved with executor.map
with ThreadPoolExecutor() as executor: # max_workers=os.process_cpu_count()
    for name, result in executor.map(_compute_split, all_dist_matrices):
        forest_amounts[name] = result
#for name, dist_matrix in all_dist_matrices:
#    name, result = _compute_split((name, dist_matrix))
#    forest_amounts[name] = result

In [34]:
{name: amt for name, amt in forest_amounts.items() if amt is not None}

{'100001': (8, 2),
 '10007': (8, 2),
 '10008': (8, 2),
 '10010': (8, 2),
 '12290': (10, 2),
 '15002': (13, 2),
 '15005': (13, 2),
 '15007': (13, 2),
 '20009': (18, 2),
 'bayg29_hard': (27, 2),
 'brazil58_hard': (56, 2),
 'd493': (1, 492),
 'pr107': (54, 53),
 'pr264': (132, 132),
 'si535': (304, 231),
 'ulysses16.tsp': (15, 1),
 'ulysses22.tsp': (21, 1)}